In [1]:
from ortools.linear_solver import pywraplp
import numpy as np
import time

load c:\Users\ADMIN\Documents\2024.2\IT4663\codeToiUu\.venv\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\ADMIN\Documents\2024.2\IT4663\codeToiUu\.venv\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\ADMIN\Documents\2024.2\IT4663\codeToiUu\.venv\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\ADMIN\Documents\2024.2\IT4663\codeToiUu\.venv\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\ADMIN\Documents\2024.2\IT4663\codeToiUu\.venv\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\ADMIN\Documents\2024.2\IT4663\codeToiUu\.venv\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\ADMIN\Documents\2024.2\IT4663\codeToiUu\.venv\Lib\site-packages\ortools\.libs\ortools.dll...


In [2]:
def read_input_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    N, D, A, B = map(int, lines[0].split())
    dayoff = np.zeros((N+1, D+1))
    for i in range(1, N+1):
        numbers = list(map(int, lines[i].split()))
        for j in range(0, len(numbers) - 1):
            dayoff[i][numbers[j]] = 1
    return N, D, A, B, dayoff

filename = 'Testcase/tc'
output = 'Output/output'

In [3]:
list_test = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
for tc in list_test:
    tmp = filename + str(tc) + '.txt'
    N, D, A, B, dayoff = read_input_from_file(tmp)

    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Biến x[i, j]
    x = {}
    for i in range(1, N+1):
        for j in range(1, D+1):
            x[i, j] = solver.IntVar(0, 4, f'x[{i},{j}]')

    # Ràng buộc 1: Mỗi nhân viên mỗi ngày làm tối đa 1 ca (đã được đảm bảo bởi định nghĩa x)

    # Ràng buộc 2: Nếu làm ca đêm hôm trước thì nghỉ hôm sau
    for i in range(1, N+1):
        for j in range(2, D+1):
            solver.Add(x[i, j] + 4 * (x[i, j - 1] - 4) <= 0)

    # Ràng buộc 3: Ngày nghỉ của nhân viên thì phải nghỉ
    for i in range(1, N+1):
        for j in range(1, D+1):
            if dayoff[i][j] == 1:
                solver.Add(x[i, j] <= 0)

    # Ràng buộc 4: Số lượng nhân viên mỗi ca (không dùng biến phụ 3 chiều)
    for d in range(1, D+1):
        for k in range(1, 5):  # Ca 1, 2, 3, 4
            # Tạo biến nhị phân tạm thời để biểu thị x[i, d] == k
            z = {}
            for i in range(1, N+1):
                z[i] = solver.BoolVar(f'z[{i},{d},{k}]')
                # Ràng buộc: z[i] = 1 nếu x[i, d] == k, z[i] = 0 nếu không
                M = 1000  # Hằng số lớn
                solver.Add(x[i, d] - k <= M * (1 - z[i]))
                solver.Add(x[i, d] - k >= -M * (1 - z[i]))
                # Ràng buộc bổ sung để đảm bảo z[i] chính xác
                solver.Add(z[i] <= (x[i, d] + (5 - k)) / k)  # Nếu x[i, d] != k, z[i] = 0

            # Tính tổng số nhân viên làm ca k ngày d
            S_kd = solver.Sum(z[i] for i in range(1, N+1))
            solver.Add(S_kd >= A)
            solver.Add(S_kd <= B)

    # Hàm mục tiêu: Tối thiểu hóa số ca đêm tối đa của mỗi nhân viên
    goal = solver.IntVar(0, D, 'goal')
    for i in range(1, N+1):
        night_shifts = []
        for j in range(1, D+1):
            night = solver.BoolVar(f'night[{i},{j}]')
            # Ràng buộc: night[i, j] = 1 nếu x[i, j] = 4, và 0 nếu không
            # Nếu x[i, j] = 4 thì night[i, j] = 1
            solver.Add(night >= (x[i, j] - 3))  # x[i, j] = 4 => night = 1, x[i, j] < 4 => night >= 0
            solver.Add(night <= (x[i, j] / 4))  # x[i, j] = 4 => night <= 1, x[i, j] < 4 => night <= 0
            night_shifts.append(night)

        # Tính tổng số ca đêm của nhân viên i
        num_night_shifts = solver.Sum(night_shifts)
        solver.Add(goal >= num_night_shifts)

    start = time.time()
    solver.Minimize(goal)
    status = solver.Solve()
    end = time.time()

    if status == pywraplp.Solver.OPTIMAL:
        print(f'Test {tc} - Optimal solution found with objective value: {goal.solution_value()}')
        # In ra ma trận x[i][j]
        for i in range(1, N+1):
            for j in range(1, D+1):
                print(f'x[{i},{j}] = {x[i,j].solution_value()}', end=' ')
            print()
        # In số lượng nhân viên mỗi ca mỗi ngày để kiểm tra
        for d in range(1, D+1):
            for k in range(1, 5):
                count = sum(1 for i in range(1, N+1) if x[i, d].solution_value() == k)
                print(f'Ngày {d}, Ca {k}: {count} nhân viên')
        # In số ca đêm của mỗi nhân viên để kiểm tra
        for i in range(1, N+1):
            night_count = sum(1 for j in range(1, D+1) if x[i, j].solution_value() == 4)
            print(f'Nhân viên {i} làm {night_count} ca đêm')
    else:
        print(f'Test {tc} - No optimal solution found.')
    print(f'Test {tc} - Time taken: {end - start:.2f} seconds')

Test 1 - Optimal solution found with objective value: 1.0
x[1,1] = 2.0 x[1,2] = 0.0 x[1,3] = 1.0 x[1,4] = 1.0 x[1,5] = 3.0 x[1,6] = 0.0 
x[2,1] = 1.0 x[2,2] = 0.0 x[2,3] = 4.0 x[2,4] = 0.0 x[2,5] = 2.0 x[2,6] = 0.0 
x[3,1] = 1.0 x[3,2] = 0.0 x[3,3] = 0.0 x[3,4] = 0.0 x[3,5] = 1.0 x[3,6] = 0.0 
x[4,1] = 4.0 x[4,2] = 0.0 x[4,3] = 2.0 x[4,4] = 0.0 x[4,5] = 0.0 x[4,6] = 1.0 
x[5,1] = 0.0 x[5,2] = 2.0 x[5,3] = 0.0 x[5,4] = 3.0 x[5,5] = 4.0 x[5,6] = 0.0 
x[6,1] = 1.0 x[6,2] = 0.0 x[6,3] = 0.0 x[6,4] = 0.0 x[6,5] = 0.0 x[6,6] = 4.0 
x[7,1] = 3.0 x[7,2] = 0.0 x[7,3] = 3.0 x[7,4] = 4.0 x[7,5] = 0.0 x[7,6] = 0.0 
x[8,1] = 0.0 x[8,2] = 1.0 x[8,3] = 0.0 x[8,4] = 2.0 x[8,5] = 0.0 x[8,6] = 2.0 
x[9,1] = 1.0 x[9,2] = 4.0 x[9,3] = 0.0 x[9,4] = 1.0 x[9,5] = 0.0 x[9,6] = 0.0 
x[10,1] = 1.0 x[10,2] = 3.0 x[10,3] = 0.0 x[10,4] = 1.0 x[10,5] = 0.0 x[10,6] = 3.0 
Ngày 1, Ca 1: 5 nhân viên
Ngày 1, Ca 2: 1 nhân viên
Ngày 1, Ca 3: 1 nhân viên
Ngày 1, Ca 4: 1 nhân viên
Ngày 2, Ca 1: 1 nhân viên
Ngày 2, Ca 2: 1 